# Exploring the FORCE 2020 Well Log Challenge - Part 3
## Interactive Map Dashboard

**Brendon Hall, Enthought**

bhall@enthought.com

In this notebook tutotorial, we'll build on the map view of the wells we made in [Part 2](https://github.com/brendonhall/FORCE-2020-Lithology/blob/master/notebooks/02-Map-View.ipynb) and create an interactive dashboard using [Plotly's Dash framework](https://dash.plotly.com/).

The really interesting part of this challenge is the fact that not all of the wells in the dataset have the same set of curves.  They all have gamma ray (GR), but otherwise each well has a subset of all possible curves.  When building models, it might be useful to be able to use subsets of the data that have the curves we want to train specific models.  Here, we'll build a tool that let's us select groups of wells based on which curves they contain, and also using the UI selection tools provided by Plotly. The tool will display the selected wells names in a list so they can be easily copy/pasted. This is a screenshot of the web app we'll build in this tutorial:

![map of wells](images/dash_example.png)

Please get in touch if you have any questions.  You can also join in the conversation on [Software Underground's slack](https://softwareunderground.org/slack) in the **#force_2020_ml_contest** channel.

Feel free to use this code, hack it, adapt it for your own needs.

The well log data is licensed as [Norwegian License for Open Government Data (NLOD) 2.0](https://data.norge.no/nlod/en/2.0/).
The well log labels that are included are provided by FORCE 2020 Machine Learning Contest under [CC-BY-4.0](https://creativecommons.org/licenses/by/4.0/).

In [2]:
!pip install jupyter-dash

     |████████████████████████████████| 7.3 MB 6.8 MB/s eta 0:00:01
     |████████████████████████████████| 23.9 MB 1.3 MB/s eta 0:00:011
     |████████████████████████████████| 357 kB 70.6 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 75.8 MB/s eta 0:00:01
  Created wheel for dash-core-components: filename=dash_core_components-2.0.0-py3-none-any.whl size=3821 sha256=ebd9812850f723a23e78fa2db114da28de2ee7a5d5ad51ad130929a92c386a3f
  Stored in directory: /root/.cache/pip/wheels/00/f9/c7/1a6437d794ed753ea9bc9079e761d4fc803a1f1f5d3697b9ec
  Created wheel for dash-html-components: filename=dash_html_components-2.0.0-py3-none-any.whl size=4089 sha256=47d96a86d623507311fd3eba60cd77977d4d8a404525d724821ab8f1ef15b14e
  Stored in directory: /root/.cache/pip/wheels/ec/6b/81/05aceabd8b27f724e2c96784016287cc1bfbc349ebfda451de
  Created wheel for dash-table: filename=dash_table-5.0.0-py3-none-any.whl size=3911 sha256=9f84bad7844f58aed88102ddfa1c777faf852b82c63473993fb8e60d30a5dbc6

In [3]:
import os.path

import numpy as np
import pandas as pd

import plotly.express as px

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

pd.options.display.max_rows = 8

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: UserWarning:


The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: UserWarning:


The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`



In the previous notebook, we grabbed well metadata from the [NPD Factpages summary page](https://factpages.npd.no/en/wellbore/tableview/exploration/all), and used that to find locations of the the wells in our dataset.  Run that notebook if you haven't already to generate the metadata file, which we load here.

In [5]:
locations_df = pd.read_csv('force_2020_meta.csv')

By now you should have local copies of the training and test datasets (see [previous notebook](https://github.com/brendonhall/FORCE-2020-Lithology/blob/master/notebooks/02-Map-View.ipynb) or the [contest website](https://xeek.ai/challenges/force-well-logs/data) if you need them).  

Here we load local copies, and combine them into a single dataframe.

In [8]:
# need to add to this, boolean values for whether a well has a value for each of the curves
local_train_csv = 'train.csv'
train_df = pd.read_csv(local_train_csv)

local_test_csv = 'test.csv'
test_df = pd.read_csv(local_test_csv)

well_data_df = pd.concat([train_df, test_df])

The dashboard we're building will allow us to filter the set of wells by the curves they contain.  We'll need a quick way to tell if a given well has a valid curve in the data.  We can use the `locations_df` to track this, with a column for each curve type and a boolean value that indicates if the log curve is present.  Here we define a list of valid `curves`, and then apply a function `well_had_curve` that checks if all values of the curve for a specific well are `nan`.  It does this for each well and every curve.  This takes a minute or so to run on my machine.  Perhaps you can think of a more efficient method?

In [10]:
curves = ['FORMATION', 'CALI', 'RSHA', 'RMED', 'RDEP', 'RHOB', 'GR', 'SGR',
          'NPHI', 'PEF', 'DTC', 'SP', 'BS', 'ROP', 'DTS', 'DCAL', 'DRHO',
          'MUDWEIGHT', 'RMIC', 'ROPA', 'RXO']

def well_has_curve(row, data_df, curve):
    well_name = row['WELL']
    return not data_df.loc[data_df['WELL']==well_name, curve].isnull().all()
    
for curve in curves:
    locations_df[curve] = locations_df.apply(lambda row: well_has_curve(row, well_data_df, curve), axis=1)

In [11]:
locations_df

,WELL,WELL_HEAD,lat,lon,Drilling Operator,Purpose,Completion Year,Formation,Dataset,FORMATION,CALI,RSHA,RMED,RDEP,RHOB,GR,SGR,NPHI,PEF,DTC,SP,BS,ROP,DTS,DCAL,DRHO,MUDWEIGHT,RMIC,ROPA,RXO
0,15/9-13,15/9-13,58.373878,1.934128,Den norske stats oljeselskap a.s,APPRAISAL,1982.0,ZECHSTEIN GP,Train,True,True,True,True,True,True,True,False,True,True,True,True,False,True,False,False,True,True,False,False,True
1,15/9-15,15/9-15,58.302069,1.922131,Den norske stats oljeselskap a.s,WILDCAT,1982.0,SKAGERRAK FM,Train,True,True,True,True,True,True,True,False,True,True,True,True,True,True,False,True,True,True,False,False,True
2,15/9-17,15/9-17,58.445608,1.948217,Den norske stats oljeselskap a.s,WILDCAT,1983.0,SMITH BANK FM,Train,True,True,True,True,True,True,True,True,True,True,True,True,False,True,False,False,True,True,False,False,True
3,16/1-2,16/1-2,58.935894,2.222239,Esso Exploration and Production Norway A/S,APPRAISAL,1976.0,BASEMENT,Train,True,True,True,True,True,True,True,False,True,False,True,True,True,True,False,False,True,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,34/3-3 A,34/3-3,61.795136,2.717883,BG Norge AS,APPRAISAL,2012.0,BURTON FM,Test,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
105,34/6-1 S,34/6-1,61.582317,2.685472,Norske Conoco A/S,WILDCAT,2002.0,LUNDE FM,Test,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
106,35/6-2 S,35/6-2,61.533606,3.911311,StatoilHydro Petroleum AS,WILDCAT,2009.0,NO FORMAL NAME,Test,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
107,35/9-8,35/9-8,61.285269,3.675594,Wintershall Norge AS,APPRAISAL,2013.0,RANNOCH FM,Test,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


Now let's use this dataframe to build a dashboard using the Dash Framework.  Dash let's you build interactive dashboard web apps using only Python - no html or css required.  If you already are using Plotly for viz then it's pretty straightforward to get started.  I'll discuss the components of the visualization below, but I encourage you to go through the [Dash tutorial docs](https://dash.plotly.com/installation) to for more insight in to how these components fit together.  It's well written and contains more than enough info to get started.

I'm also using the [`jupyter-dash`](https://github.com/plotly/jupyter-dash) package to render the dashboard right here in the notebook.  This is useful in a tutorial context, but also for testing out small components.  

The first thing we do is create the `JupyterDash` object.  Then we describe the layout of the app.  Dash provides Python objects that wrap the various `html` elements that comprise the web interface.  We're going to create a simple UI to start.  In addition to some titles and headings, we use a [multi-value dropdown](https://dash.plotly.com/dash-core-components/dropdown) to indicate which curves are required for the well selection.  We'll also use a [checklist](https://dash.plotly.com/dash-core-components/checklist) to indicate whether to include the testing and training data (or both).  A graph object will display the map.  Finally we include a code text section that will display a list of the names of the selected wells.

In [12]:
# use Jupyter Dash to quickly build a prototype of the dash app right in this notebook
# for production or more complicated use we'll use the webserver version
app = JupyterDash(__name__)

app.layout = html.Div([
    html.H2("FORCE 2020 Well Data Map"),
    html.Label([
        "Selected wells must contain:",
        dcc.Dropdown(
            id='curves-dropdown', clearable=False,
            options=[
                {'label': c, 'value': c}
                for c in curves
            ],
            value=['GR'],
            multi=True),
        dcc.Checklist(
            id='datasets-checkbox',
        options=[
            {'label': 'Training Data', 'value': 'Train'},
            {'label': 'Test Data', 'value': 'Test'}
            ],
            value=['Train', 'Test']),  
        dcc.Graph(id='map-plot'),
        
        # code-styled text area to display list of filtered well names
        html.Code(id='selected-wells-output', style={'whiteSpace': 'pre-line'})
    ]),   
])

Now that we have defined the layout, we need to let Dash know how to render the map plot based on the user input.  This is done by defining a [callback function](https://dash.plotly.com/basic-callbacks), which is called when the specified UI elements are activated.  Callbacks provide the interactivity of the dashboard.

A function decorator is used to indicate what UI elements this callback listens to, and what `Graph` element is updated.  The map plot we're making here is the same as the one from the previous notebook.  We just need to filter the dataframe and remove the wells that don't contain the proper curves, and the testing/training data as indicated by the UI.

In [13]:
# Define callback to update graph
@app.callback(
    Output('map-plot', 'figure'),
    [Input("curves-dropdown", "value"),
     Input('datasets-checkbox', 'value')]
)
def update_figure(curves, datasets):

    curve_mask = locations_df[curves].values.all(axis=1)
    dff = locations_df[curve_mask]
    
    dff = dff[dff['Dataset'].isin(datasets)]
    
    num_wells = dff.shape[0]
    
    # if the selection is empty, just display the map centered at the 
    # same location
    if num_wells == 0:
        fig = px.scatter_mapbox(lat=[59.610511], lon= [2.772594],
                                center={'lat': 59.610511, 'lon': 2.772594},
                                zoom=5, height=600,
                                )

    else:
        fig = px.scatter_mapbox(dff, lat="lat", lon="lon",
                                center={'lat': 59.610511, 'lon': 2.772594},
                                color='Dataset',
                                zoom=5, height=600,
                                hover_data={'WELL': True,
                                            'lat': False,
                                            'lon': False,
                                            'Dataset': False,
                                            'Drilling Operator': True,
                                            'Purpose': True,
                                            'Completion Year': True,
                                            'Formation': True},
                                color_discrete_map={'Train': 'blue',
                                                    'Test': 'red'}
                                )

    fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(margin={"r":40,"t":50,"l":40,"b":0})

    return fig

I thought it would be useful to provide a Python style list of the filtered well names.  This text could be copied and pasted directly into a notebook or script to get a subset of wells to train models.  The code below constructs this list based on the curve and test/train UI elements.  It will also respond to selections made using the lasso and square selection tools provided by the Plotly map.

In [14]:
@app.callback(
    Output('selected-wells-output', 'children'),
    [Input("curves-dropdown", "value"),
     Input('datasets-checkbox', 'value'),
     Input('map-plot', 'selectedData')]
)
def update_output(curves, datasets, selected_items):
    
    # return the well names for all wells that match the criterion
    curve_mask = locations_df[curves].values.all(axis=1)
    dff = locations_df[curve_mask]
    dff = dff[dff['Dataset'].isin(datasets)]
    # now filter by wells that have been selected in the image.
    all_wells = dff['WELL'] # initialize to all of the wells
    lasso_wells = None
    
    # now iterate over selected items, and get the relevant point data
    # need to do some checks to make sure valid data selected
    if selected_items is not None:
        for selected_item in [selected_items]:
            if selected_item and selected_item['points']:
                # get a list of the selected well names
                lasso_wells = [p['customdata'][0] for p in selected_item['points']]
   
    select_mask = dff['WELL'].isin(lasso_wells if lasso_wells else all_wells)
    dff = dff[select_mask]
    
    num_wells = dff.shape[0]
    well_names = dff['WELL'].values

    output_text = "\nYou have selected {} wells.\n\n".format(num_wells)
    well_list_text = ", ".join("'{}'".format(well_name) for well_name in well_names)
    well_list_text = "selected_wells = ["+well_list_text + "]"
    return output_text + well_list_text



That's it!  Now we launch the web app.  JupyterDash lets you do this in a couple different ways.  You can render the app directly in the notebook, or launch a separate browser tab.

In [19]:
# run the line below to render the dashboard right in the notebook
app.css.config.serve_locally = True
app.run_server(mode='inline', debug=True)

# use this to render in a separate browser tab
# app.run_server(mode='external')

<IPython.core.display.Javascript object>

In [20]:
app.run_server(mode='inline', debug=True, host="0.0.0.0",port=1005)

<IPython.core.display.Javascript object>

Test out the application, see what you think.  Did you find it useful? Can you think of any features you'd like to add?  Are there any bugs?  Please share your thoughts!  


This notebook is open source content. Text is CC-BY-4.0, code is [Apache 2.0](https://www.apache.org/licenses/LICENSE-2.0).

### References

Bormann P., Aursand P., Dilib F., Dischington P., Manral S. (2020) 2020 FORCE Machine Learning Contest. https://github.com/bolgebrygg/Force-2020-Machine-Learning-competition